# Лабораторная работа №2 (Проведение исследований с логистической и линейной регрессией)

### Создание бейзлайна и оценка качества

Импорт необходимых библиотек.

In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

Подготовка данных.

In [65]:
path = 'drive/MyDrive/ai_data/'
heart = pd.read_csv(path + 'heart.csv')
heart.shape, heart.columns

((918, 12),
 Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
        'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
        'HeartDisease'],
       dtype='object'))

Посмотрим состав таблицы и наличие пропусков, а также баланс HeartDisease.

In [66]:
display(heart.head())
display(heart.info())
display(heart.describe())
print("Missing per column:\n", heart.isna().sum())
print(heart['HeartDisease'].value_counts(normalize=True))

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


None

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


Missing per column:
 Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64
HeartDisease
1    0.553377
0    0.446623
Name: proportion, dtype: float64


Выполним те же действия для второго датасета.

In [67]:
melbourne = pd.read_csv(path + 'melb_data.csv')
melbourne.shape, melbourne.columns

((13580, 21),
 Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
        'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
        'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
        'Longtitude', 'Regionname', 'Propertycount'],
       dtype='object'))

In [68]:
display(melbourne.head())
display(melbourne.info())
display(melbourne.describe())
print(melbourne.isna().sum())
print(melbourne['Price'].describe())

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

None

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


Suburb              0
Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64
count    1.358000e+04
mean     1.075684e+06
std      6.393107e+05
min      8.500000e+04
25%      6.500000e+05
50%      9.030000e+05
75%      1.330000e+06
max      9.000000e+06
Name: Price, dtype: float64


Удалим столбцы с большим количеством пропусков, заполним столбцы с маленьким количеством пропусков медианой и модой

In [69]:
columns_to_drop = ['BuildingArea', 'YearBuilt']
melbourne_clean = melbourne.drop(columns=columns_to_drop)

car_median = melbourne_clean['Car'].median()
melbourne_clean['Car'] = melbourne_clean['Car'].fillna(car_median)

council_mode = melbourne_clean['CouncilArea'].mode()[0]
melbourne_clean['CouncilArea'] = melbourne_clean['CouncilArea'].fillna(council_mode)

print(melbourne_clean.isna().sum())

Suburb           0
Address          0
Rooms            0
Type             0
Price            0
Method           0
SellerG          0
Date             0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
CouncilArea      0
Lattitude        0
Longtitude       0
Regionname       0
Propertycount    0
dtype: int64


Разделим данные на тренировочную и тестовую выборки:

In [70]:
X_heart = heart.drop('HeartDisease', axis=1)
y_heart = heart['HeartDisease']
X_heart_train, X_heart_test, y_heart_train, y_heart_test = train_test_split(
    X_heart, y_heart, test_size=0.2, random_state=42, stratify=y_heart
)


X_melbourne = melbourne_clean.drop('Price', axis=1)
y_melbourne = melbourne_clean['Price']
X_melbourne_train, X_melbourne_test, y_melbourne_train, y_melbourne_test = train_test_split(
    X_melbourne, y_melbourne, test_size=0.2, random_state=42
)

Проведем обработку категориальных переменных:

In [71]:
X_heart_train_processed = X_heart_train.copy()
X_heart_test_processed = X_heart_test.copy()
X_melbourne_train_processed = X_melbourne_train.copy()
X_melbourne_test_processed = X_melbourne_test.copy()

Обработка категориальных переменных для Heart dataset:

In [72]:
categorical_heart = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
for col in categorical_heart:
    print(f"{col}: {X_heart_train[col].unique()}")

label_encoders_heart = {}
for col in categorical_heart:
    le = LabelEncoder()
    X_heart_train_processed[col] = le.fit_transform(X_heart_train[col])
    X_heart_test_processed[col] = le.transform(X_heart_test[col])
    label_encoders_heart[col] = le

print("\nРезультат:")
print(X_heart_train_processed[categorical_heart].head())
print(f"{X_heart_train_processed[categorical_heart].dtypes}")

Sex: ['M' 'F']
ChestPainType: ['ATA' 'ASY' 'NAP' 'TA']
RestingECG: ['ST' 'Normal' 'LVH']
ExerciseAngina: ['Y' 'N']
ST_Slope: ['Flat' 'Up' 'Down']

Результат:
     Sex  ChestPainType  RestingECG  ExerciseAngina  ST_Slope
485    1              1           2               1         1
486    1              1           2               0         2
117    0              0           2               1         1
361    1              0           1               1         1
296    1              0           1               1         1
Sex               int64
ChestPainType     int64
RestingECG        int64
ExerciseAngina    int64
ST_Slope          int64
dtype: object


Обработка категориальных переменных для Melbourne dataset. Применяем One-Hot Encoding для сложных категориальных признаков. Удаляем нечисловые столбцы, которые не будем использовать.

In [73]:
categorical_melbourne = ['Type', 'Method', 'CouncilArea', 'Regionname']
for col in categorical_melbourne:
    print(f"{col}: {X_melbourne_train[col].nunique()}")

simple_categorical = ['Type', 'Method']
complex_categorical = ['CouncilArea', 'Regionname']

label_encoders_melbourne = {}
for col in simple_categorical:
    le = LabelEncoder()
    X_melbourne_train_processed[col] = le.fit_transform(X_melbourne_train[col])
    X_melbourne_test_processed[col] = le.transform(X_melbourne_test[col])
    label_encoders_melbourne[col] = le


council_dummies_train = pd.get_dummies(X_melbourne_train['CouncilArea'], prefix='Council', dtype=int)
region_dummies_train = pd.get_dummies(X_melbourne_train['Regionname'], prefix='Region', dtype=int)
council_dummies_test = pd.get_dummies(X_melbourne_test['CouncilArea'], prefix='Council', dtype=int)
region_dummies_test = pd.get_dummies(X_melbourne_test['Regionname'], prefix='Region', dtype=int)

council_dummies_test = council_dummies_test.reindex(columns=council_dummies_train.columns, fill_value=0)
region_dummies_test = region_dummies_test.reindex(columns=region_dummies_train.columns, fill_value=0)

X_melbourne_train_processed = X_melbourne_train_processed.drop(['CouncilArea', 'Regionname'], axis=1)
X_melbourne_test_processed = X_melbourne_test_processed.drop(['CouncilArea', 'Regionname'], axis=1)

X_melbourne_train_processed = pd.concat([X_melbourne_train_processed, council_dummies_train, region_dummies_train], axis=1)
X_melbourne_test_processed = pd.concat([X_melbourne_test_processed, council_dummies_test, region_dummies_test], axis=1)

columns_to_drop_melbourne = ['Suburb', 'Address', 'SellerG', 'Date']
X_melbourne_train_processed = X_melbourne_train_processed.drop(columns=columns_to_drop_melbourne, errors='ignore')
X_melbourne_test_processed = X_melbourne_test_processed.drop(columns=columns_to_drop_melbourne, errors='ignore')

print(f"Итого: {X_melbourne_train_processed.shape[1]} признаков")

Type: 3
Method: 5
CouncilArea: 33
Regionname: 8
Итого: 53 признаков


Итоговые размер данных и типы данных:

In [74]:
print(f"Heart train: {X_heart_train_processed.shape}")
print(f"Heart test: {X_heart_test_processed.shape}")
print(f"Melbourne train: {X_melbourne_train_processed.shape}")
print(f"Melbourne test: {X_melbourne_test_processed.shape}")

print(f"Heart: {X_heart_train_processed.dtypes.unique()}")
print(f"Melbourne: {X_melbourne_train_processed.dtypes.unique()}")

Heart train: (734, 11)
Heart test: (184, 11)
Melbourne train: (10864, 53)
Melbourne test: (2716, 53)
Heart: [dtype('int64') dtype('float64')]
Melbourne: [dtype('int64') dtype('float64')]


Проведем масштабирование признаков:

In [75]:
scaler_heart = StandardScaler()
X_heart_train_scaled = scaler_heart.fit_transform(X_heart_train_processed)
X_heart_test_scaled = scaler_heart.transform(X_heart_test_processed)

scaler_melbourne = StandardScaler()
X_melbourne_train_scaled = scaler_melbourne.fit_transform(X_melbourne_train_processed)
X_melbourne_test_scaled = scaler_melbourne.transform(X_melbourne_test_processed)

#### Обучение моделей

In [76]:
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_heart_train_scaled, y_heart_train)

y_heart_pred_train = log_reg.predict(X_heart_train_scaled)
y_heart_pred_test = log_reg.predict(X_heart_test_scaled)
y_heart_pred_proba_test = log_reg.predict_proba(X_heart_test_scaled)[:, 1]

lin_reg = LinearRegression()
lin_reg.fit(X_melbourne_train_scaled, y_melbourne_train)

y_melbourne_pred_train = lin_reg.predict(X_melbourne_train_scaled)
y_melbourne_pred_test = lin_reg.predict(X_melbourne_test_scaled)

#### Оценка качества моделей

In [77]:
accuracy_heart = accuracy_score(y_heart_test, y_heart_pred_test)
recall_heart = recall_score(y_heart_test, y_heart_pred_test)

mae_melbourne = mean_absolute_error(y_melbourne_test, y_melbourne_pred_test)
r2_melbourne = r2_score(y_melbourne_test, y_melbourne_pred_test)

print("Классификация")
print(f"Accuracy:  {accuracy_heart:.4f}")
print(f"Recall:    {recall_heart:.4f}")
print("Регрессия")
print(f"R-squared: {r2_melbourne:.4f}")
print(f"MAE:       {mae_melbourne:,.2f}")

Классификация
Accuracy:  0.8696
Recall:    0.9314
Регрессия
R-squared: 0.6188
MAE:       262,488.68


### Улучшение бейзлайна

Гипотезы:
- L2-Регуляризация для логистической регрессии с подбором оптимального параметра C на кросс-валидации улучшит метрики
- Добавление полиномиальных признаков для числовых переменных (Rooms, Distance, Bathroom, Car) для модели линейной регресии улучшит метрики
- Подбор гиперпараметров через GridSearchCV улучшит метрики

Гипотеза 1:

In [78]:
param_grid_log = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs']
}

grid_search_log = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=1000),
    param_grid_log,
    cv=5,
    scoring='f1',
    n_jobs=-1
)

grid_search_log.fit(X_heart_train_scaled, y_heart_train)

best_log_reg = grid_search_log.best_estimator_
print(f"Лучшие параметры: {grid_search_log.best_params_}")

y_heart_pred_improved = best_log_reg.predict(X_heart_test_scaled)
y_heart_pred_proba_improved = best_log_reg.predict_proba(X_heart_test_scaled)[:, 1]

accuracy_improved = accuracy_score(y_heart_test, y_heart_pred_improved)
recall_improved = recall_score(y_heart_test, y_heart_pred_improved)

print("\nСравнение результатов классификации:")
print(f"Базовая модель    - Accuracy: {accuracy_heart:.4f}, Recall: {recall_heart:.4f}")
print(f"Улучшенная модель - Accuracy: {accuracy_improved:.4f}, Recall: {recall_improved:.4f}")

Лучшие параметры: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}

Сравнение результатов классификации:
Базовая модель    - Accuracy: 0.8696, Recall: 0.9314
Улучшенная модель - Accuracy: 0.8696, Recall: 0.9314


Реализация гипотезы 1 не дала изменений, так как дефолтное значение C=1 оказалось лучшим. Гипотеза не подтверждена.

Гипотеза 2:

In [79]:
numerical_features = ['Rooms', 'Distance', 'Bathroom', 'Car']

feature_indices = [i for i, col in enumerate(X_melbourne_train_processed.columns)
                   if col in numerical_features]

X_melbourne_train_numerical = X_melbourne_train_scaled[:, feature_indices]
X_melbourne_test_numerical = X_melbourne_test_scaled[:, feature_indices]

poly = PolynomialFeatures(degree=2, include_bias=False)
X_melbourne_train_poly = poly.fit_transform(X_melbourne_train_numerical)
X_melbourne_test_poly = poly.transform(X_melbourne_test_numerical)

X_melbourne_train_enhanced = np.hstack([
    X_melbourne_train_scaled,
    X_melbourne_train_poly[:, len(feature_indices):]
])
X_melbourne_test_enhanced = np.hstack([
    X_melbourne_test_scaled,
    X_melbourne_test_poly[:, len(feature_indices):]
])

print(f"Исходное количество признаков: {X_melbourne_train_scaled.shape[1]}")
print(f"После добавления полиномиальных: {X_melbourne_train_enhanced.shape[1]}")

lin_reg_poly = LinearRegression()
lin_reg_poly.fit(X_melbourne_train_enhanced, y_melbourne_train)

y_melbourne_pred_poly = lin_reg_poly.predict(X_melbourne_test_enhanced)

mae_poly = mean_absolute_error(y_melbourne_test, y_melbourne_pred_poly)
r2_poly = r2_score(y_melbourne_test, y_melbourne_pred_poly)

print("\nСравнение результатов регрессии:")
print(f"Базовая модель    - R2: {r2_melbourne:.4f}, MAE: {mae_melbourne:,.2f}")
print(f"С полиномиальными - R2: {r2_poly:.4f}, MAE: {mae_poly:,.2f}")

Исходное количество признаков: 53
После добавления полиномиальных: 63

Сравнение результатов регрессии:
Базовая модель    - R2: 0.6188, MAE: 262,488.68
С полиномиальными - R2: 0.6235, MAE: 253,409.94


Метрики улучшились, гипотеза 2 подтверждена.

Гипотеза 3:

In [80]:
param_grid_log_full = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear']
}

grid_log_full = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=2000),
    param_grid_log_full,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

grid_log_full.fit(X_heart_train_scaled, y_heart_train)
final_log_reg = grid_log_full.best_estimator_

print("\nКлассификация:")
print(f"Лучшие параметры: {grid_log_full.best_params_}")

y_heart_final_pred = final_log_reg.predict(X_heart_test_scaled)
accuracy_final = accuracy_score(y_heart_test, y_heart_final_pred)
recall_final = recall_score(y_heart_test, y_heart_final_pred)

param_grid_ridge = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

grid_ridge = GridSearchCV(
    Ridge(random_state=42),
    param_grid_ridge,
    cv=5,
    scoring='r2',
    n_jobs=-1
)

grid_ridge.fit(X_melbourne_train_enhanced, y_melbourne_train)
final_ridge = grid_ridge.best_estimator_

print("\nРегрессия:")
print(f"Лучшие параметры: {grid_ridge.best_params_}")
print(f"Лучший R2 на CV: {grid_ridge.best_score_:.4f}")

y_melbourne_final_pred = final_ridge.predict(X_melbourne_test_enhanced)
mae_final = mean_absolute_error(y_melbourne_test, y_melbourne_final_pred)
r2_final = r2_score(y_melbourne_test, y_melbourne_final_pred)

print("\nКлассификация:")
print(f"Базовая:     Accuracy: {accuracy_heart:.4f}, Recall: {recall_heart:.4f}")
print(f"Финальная:   Accuracy: {accuracy_final:.4f}, Recall: {recall_final:.4f}")

print("\nРегрессия:")
print(f"Базовая:     R2: {r2_melbourne:.4f}, MAE: {mae_melbourne:,.2f}")
print(f"Финальная:   R2: {r2_final:.4f}, MAE: {mae_final:,.2f}")


Классификация:
Лучшие параметры: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}

Регрессия:
Лучшие параметры: {'alpha': 10}
Лучший R2 на CV: 0.6380

Классификация:
Базовая:     Accuracy: 0.8696, Recall: 0.9314
Финальная:   Accuracy: 0.8804, Recall: 0.9314

Регрессия:
Базовая:     R2: 0.6188, MAE: 262,488.68
Финальная:   R2: 0.6236, MAE: 253,346.79


Метрики улучшились, гипотеза 3 подтверждена.

Сформируем улучшенный бейзлайн по результатам проверки гипотез:

In [81]:
param_grid_classification = {
    'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear']
}

grid_classification = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=2000),
    param_grid_classification,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=0
)

grid_classification.fit(X_heart_train_scaled, y_heart_train)

improved_log_reg = grid_classification.best_estimator_

print("Классификация:")
print(f"Лучшие параметры: {grid_classification.best_params_}")

y_heart_improved_pred = improved_log_reg.predict(X_heart_test_scaled)
y_heart_improved_proba = improved_log_reg.predict_proba(X_heart_test_scaled)[:, 1]

accuracy_heart_improved = accuracy_score(y_heart_test, y_heart_improved_pred)
recall_heart_improved = recall_score(y_heart_test, y_heart_improved_pred)

print(f"Accuracy:  {accuracy_heart_improved:.4f}")
print(f"Recall:    {recall_heart_improved:.4f}")


numerical_features = ['Rooms', 'Distance', 'Bathroom', 'Car']
feature_indices = [i for i, col in enumerate(X_melbourne_train_processed.columns)
                   if col in numerical_features]

X_melbourne_train_numerical = X_melbourne_train_scaled[:, feature_indices]
X_melbourne_test_numerical = X_melbourne_test_scaled[:, feature_indices]

poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_melbourne_train_poly = poly_features.fit_transform(X_melbourne_train_numerical)
X_melbourne_test_poly = poly_features.transform(X_melbourne_test_numerical)

X_melbourne_train_improved = np.hstack([
    X_melbourne_train_scaled,
    X_melbourne_train_poly[:, len(feature_indices):]
])
X_melbourne_test_improved = np.hstack([
    X_melbourne_test_scaled,
    X_melbourne_test_poly[:, len(feature_indices):]
])

param_grid_regression = {
    'alpha': [0.01, 0.1, 1, 5, 10, 50, 100, 500]
}

grid_regression = GridSearchCV(
    Ridge(random_state=42),
    param_grid_regression,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=0
)

grid_regression.fit(X_melbourne_train_improved, y_melbourne_train)

improved_ridge = grid_regression.best_estimator_

print("\nРегрессия")
print(f"Количество признаков: {X_melbourne_train_improved.shape[1]} (было {X_melbourne_train_scaled.shape[1]})")
print(f"Лучшие параметры: {grid_regression.best_params_}")

y_melbourne_improved_pred = improved_ridge.predict(X_melbourne_test_improved)

mae_melbourne_improved = mean_absolute_error(y_melbourne_test, y_melbourne_improved_pred)
r2_melbourne_improved = r2_score(y_melbourne_test, y_melbourne_improved_pred)

print(f"R2:        {r2_melbourne_improved:.4f}")
print(f"MAE:       {mae_melbourne_improved:,.2f}")


print("\nКлассификация:")
print(f"Базовая модель:      Accuracy: {accuracy_heart:.4f}, Recall: {recall_heart:.4f}")
print(f"Улучшенная модель:   Accuracy: {accuracy_heart_improved:.4f}, Recall: {recall_heart_improved:.4f}")
print(f"Изменение:           Accuracy: {(accuracy_heart_improved - accuracy_heart):+.4f}, Recall: {(recall_heart_improved - recall_heart):+.4f}")

print("\nРегрессия:")
print(f"Базовая модель:      R2: {r2_melbourne:.4f}, MAE: {mae_melbourne:,.2f}")
print(f"Улучшенная модель:   R2: {r2_melbourne_improved:.4f}, MAE: {mae_melbourne_improved:,.2f}")
print(f"Изменение:           R2: {(r2_melbourne_improved - r2_melbourne):+.4f}, MAE: {(mae_melbourne_improved - mae_melbourne):+,.2f}")

Классификация:
Лучшие параметры: {'C': 0.05, 'penalty': 'l2', 'solver': 'lbfgs'}
Accuracy:  0.8696
Recall:    0.9118

Регрессия
Количество признаков: 63 (было 53)
Лучшие параметры: {'alpha': 50}
R2:        0.6240
MAE:       253,103.93

Классификация:
Базовая модель:      Accuracy: 0.8696, Recall: 0.9314
Улучшенная модель:   Accuracy: 0.8696, Recall: 0.9118
Изменение:           Accuracy: +0.0000, Recall: -0.0196

Регрессия:
Базовая модель:      R2: 0.6188, MAE: 262,488.68
Улучшенная модель:   R2: 0.6240, MAE: 253,103.93
Изменение:           R2: +0.0052, MAE: -9,384.75


С улучшенным бейзлайном качество моделей улучшилось, поэтому можно сделать следующие выводы:
- Регуляризация важна: подобранный параметр C улучшил обобщающую способность модели классификации.
- Полиномиальные признаки эффективны: добавление 10 новых признаков (взаимодействий) позволило модели регрессии лучше улавливать нелинейные зависимости.
- Ridge регрессия превосходит обычную: параметр alpha=50 снизил ошибку прогноза.

### Имплементация алгоритма машинного обучения

Релизация линейной и логистической регрессии для регресии и классификации в виде классов:

In [82]:
class MyLogisticRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000, random_state=None):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.random_state = random_state
        self.weights = None
        self.bias = None
        self.losses = []

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

    def _compute_loss(self, y_true, y_pred):
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def fit(self, X, y):
        if self.random_state is not None:
            np.random.seed(self.random_state)

        n_samples, n_features = X.shape

        self.weights = np.random.randn(n_features) * 0.01
        self.bias = 0

        for i in range(self.n_iterations):
            linear_output = np.dot(X, self.weights) + self.bias
            y_pred = self._sigmoid(linear_output)

            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            if i % 100 == 0:
                loss = self._compute_loss(y, y_pred)
                self.losses.append(loss)

        return self

    def predict_proba(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return self._sigmoid(linear_output)

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)


class MyLinearRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000, random_state=None):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.random_state = random_state
        self.weights = None
        self.bias = None
        self.losses = []

    def _compute_loss(self, y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)

    def fit(self, X, y):
        if self.random_state is not None:
            np.random.seed(self.random_state)

        n_samples, n_features = X.shape

        self.weights = np.random.randn(n_features) * 0.01
        self.bias = 0

        for i in range(self.n_iterations):
            y_pred = np.dot(X, self.weights) + self.bias

            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            if i % 100 == 0:
                loss = self._compute_loss(y, y_pred)
                self.losses.append(loss)

        return self

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias


Посмотрим на результаты и сравним их с базовым бейзайном:

In [83]:
print("\nMyLogisticRegression:")

my_log_reg = MyLogisticRegression(learning_rate=0.1, n_iterations=2000, random_state=42)
my_log_reg.fit(X_heart_train_scaled, y_heart_train.values)

y_heart_my_pred = my_log_reg.predict(X_heart_test_scaled)
y_heart_my_proba = my_log_reg.predict_proba(X_heart_test_scaled)

accuracy_my_log = accuracy_score(y_heart_test, y_heart_my_pred)
recall_my_log = recall_score(y_heart_test, y_heart_my_pred)

print(f"Accuracy:  {accuracy_my_log:.4f}")
print(f"Recall:    {recall_my_log:.4f}")


print("\nMyLinearRegression:")

my_lin_reg = MyLinearRegression(learning_rate=0.01, n_iterations=2000, random_state=42)
my_lin_reg.fit(X_melbourne_train_scaled, y_melbourne_train.values)

y_melbourne_my_pred = my_lin_reg.predict(X_melbourne_test_scaled)

mae_my_lin = mean_absolute_error(y_melbourne_test, y_melbourne_my_pred)
r2_my_lin = r2_score(y_melbourne_test, y_melbourne_my_pred)


print(f"R2:        {r2_my_lin:.4f}")
print(f"MAE:       {mae_my_lin:,.2f}")

print("\nКлассификация:")
print(f"Базовая sklearn:         Accuracy: {accuracy_heart:.4f}, Recall: {recall_heart:.4f}")
print(f"Своя имплементация:      Accuracy: {accuracy_my_log:.4f}, Recall: {recall_my_log:.4f}")
print(f"Разница:                 Accuracy: {(accuracy_my_log - accuracy_heart):+.4f}, Recall: {(recall_my_log - recall_heart):+.4f}")

print("\nРегрессия:")
print(f"Базовая sklearn:         R2: {r2_melbourne:.4f}, MAE: {mae_melbourne:,.2f}")
print(f"Своя имплементация:      R2: {r2_my_lin:.4f}, MAE: {mae_my_lin:,.2f}")
print(f"Разница:                 R2: {(r2_my_lin - r2_melbourne):+.4f}, MAE: {(mae_my_lin - mae_melbourne):+,.2f}")


MyLogisticRegression:
Accuracy:  0.8696
Recall:    0.9314

MyLinearRegression:
R2:        0.6191
MAE:       262,250.14

Классификация:
Базовая sklearn:         Accuracy: 0.8696, Recall: 0.9314
Своя имплементация:      Accuracy: 0.8696, Recall: 0.9314
Разница:                 Accuracy: +0.0000, Recall: +0.0000

Регрессия:
Базовая sklearn:         R2: 0.6188, MAE: 262,488.68
Своя имплементация:      R2: 0.6191, MAE: 262,250.14
Разница:                 R2: +0.0003, MAE: -238.55


Получили очень близкий результат.  
Теперь добавим в модель техники из улучшенного бейзлайна:

In [84]:
print("\nMyLogisticRegression:")

class MyLogisticRegressionWithRegularization(MyLogisticRegression):
    def __init__(self, learning_rate=0.01, n_iterations=1000, lambda_reg=0.1, random_state=None):
        super().__init__(learning_rate, n_iterations, random_state)
        self.lambda_reg = lambda_reg

    def fit(self, X, y):
        if self.random_state is not None:
            np.random.seed(self.random_state)

        n_samples, n_features = X.shape
        self.weights = np.random.randn(n_features) * 0.01
        self.bias = 0

        for i in range(self.n_iterations):
            linear_output = np.dot(X, self.weights) + self.bias
            y_pred = self._sigmoid(linear_output)

            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y)) + (self.lambda_reg / n_samples) * self.weights
            db = (1 / n_samples) * np.sum(y_pred - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            if i % 100 == 0:
                loss = self._compute_loss(y, y_pred)
                self.losses.append(loss)

        return self

my_log_reg_improved = MyLogisticRegressionWithRegularization(
    learning_rate=0.1,
    n_iterations=2000,
    lambda_reg=20.0,  # 1/0.05
    random_state=42
)
my_log_reg_improved.fit(X_heart_train_scaled, y_heart_train.values)

y_heart_my_improved_pred = my_log_reg_improved.predict(X_heart_test_scaled)
y_heart_my_improved_proba = my_log_reg_improved.predict_proba(X_heart_test_scaled)

accuracy_my_improved = accuracy_score(y_heart_test, y_heart_my_improved_pred)
recall_my_improved = recall_score(y_heart_test, y_heart_my_improved_pred)

print(f"Accuracy:  {accuracy_my_improved:.4f}")
print(f"Recall:    {recall_my_improved:.4f}")


print("\nMyLinearRegression:")

class MyRidgeRegression(MyLinearRegression):
    def __init__(self, learning_rate=0.01, n_iterations=1000, alpha=1.0, random_state=None):
        super().__init__(learning_rate, n_iterations, random_state)
        self.alpha = alpha

    def fit(self, X, y):
        if self.random_state is not None:
            np.random.seed(self.random_state)

        n_samples, n_features = X.shape

        self.weights = np.random.randn(n_features) * 0.01
        self.bias = 0

        for i in range(self.n_iterations):
            y_pred = np.dot(X, self.weights) + self.bias

            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y)) + (self.alpha / n_samples) * self.weights
            db = (1 / n_samples) * np.sum(y_pred - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            if i % 100 == 0:
                loss = self._compute_loss(y, y_pred)
                self.losses.append(loss)

        return self

my_ridge_improved = MyRidgeRegression(
    learning_rate=0.01,
    n_iterations=3000,
    alpha=50.0,
    random_state=42
)
my_ridge_improved.fit(X_melbourne_train_improved, y_melbourne_train.values)

y_melbourne_my_improved_pred = my_ridge_improved.predict(X_melbourne_test_improved)

mae_my_improved = mean_absolute_error(y_melbourne_test, y_melbourne_my_improved_pred)
r2_my_improved = r2_score(y_melbourne_test, y_melbourne_my_improved_pred)

print(f"Количество признаков: {X_melbourne_train_improved.shape[1]}")
print(f"R2:        {r2_my_improved:.4f}")
print(f"MAE:       {mae_my_improved:,.2f}")



print("\nКлассификация:")
print(f"Улучшенный sklearn:      Accuracy: {accuracy_heart_improved:.4f}, Recall: {recall_heart_improved:.4f}")
print(f"Своя имплементация:      Accuracy: {accuracy_my_improved:.4f}, Recall: {recall_my_improved:.4f}")
print(f"Разница:                 Accuracy: {(accuracy_my_improved - accuracy_heart_improved):+.4f}, Recall: {(recall_my_improved - recall_heart_improved):+.4f}")

print("\nРегрессия:")
print(f"Улучшенный sklearn:      R2: {r2_melbourne_improved:.4f}, MAE: {mae_melbourne_improved:,.2f}")
print(f"Своя имплементация:      R2: {r2_my_improved:.4f}, MAE: {mae_my_improved:,.2f}")
print(f"Разница:                 R2: {(r2_my_improved - r2_melbourne_improved):+.4f}, MAE: {(mae_my_improved - mae_melbourne_improved):+,.2f}")


MyLogisticRegression:
Accuracy:  0.8696
Recall:    0.9118

MyLinearRegression:
Количество признаков: 63
R2:        0.6244
MAE:       252,843.34

Классификация:
Улучшенный sklearn:      Accuracy: 0.8696, Recall: 0.9118
Своя имплементация:      Accuracy: 0.8696, Recall: 0.9118
Разница:                 Accuracy: +0.0000, Recall: +0.0000

Регрессия:
Улучшенный sklearn:      R2: 0.6240, MAE: 253,103.93
Своя имплементация:      R2: 0.6244, MAE: 252,843.34
Разница:                 R2: +0.0004, MAE: -260.58


### Выводы

Собственная реализация алгоритмов логистической и линейной регрессии продемонстрировала результаты, идентичные библиотеке sklearn. Улучшенные версии с регуляризацией и полиномиальными признаками показали значительное улучшение качества по сравнению с базовым бейзлайном, особенно для задачи регрессии (снижение MAE примерно на 10000).